Cargar los archivos CSV

In [146]:
import pandas as pd

# Cargar los archivos CSV
calificaciones_df = pd.read_csv('input/calificaciones.csv')
carrera_df = pd.read_csv('input/carrera.csv')
escuelas_df = pd.read_csv('input/escuelas.csv')
estudiantes_df = pd.read_csv('input/estudiantes.csv')

# Ver las primeras filas de cada DataFrame
print(calificaciones_df.head())
print(carrera_df.head())
print(escuelas_df.head())
print(estudiantes_df.head())

   periodo  clave_alumno clave_materia  calificacion  tipo_calificacion
0     2001             0       ACB9309         100.0                  1
1     2001             0       ACH9322          75.0                  1
2     2001             0       ACM9313           0.0                  3
3     2001             0       ACM9301           0.0                  3
4     2001             0       ACC9326          93.0                  1
   clave_carrera                  nombre        corto
0              0   INGENIERIA INDUSTRIAL   Industrial
1              1   INGENIERIA BIOQUIMICA   Bioquimica
2              2     INGENIERIA MECANICA     Mecanica
3              3    INGENIERIA ELECTRICA    Electrica
4              4  INGENIERIA ELECTRONICA  Electronica
   clave                         escuela
0      3  COBAES 3(PROFRA. VELINA LEON )
1      4  COBAES 4(PROFR. VICTOR M. RUB)
2      5  COBAES 5(PROFRA. FCA. GUERRER)
3      6  COBAES 6(ING.JOSE HERNANDEZ T)
4      7  COBAES 7(GRAÑ. PABLO MACIAS V

Explorar los datos
Verificamos si los datos contienen valores nulos, duplicados, o alguna inconsistencia:

In [147]:
# Verificar si hay valores nulos
print("Valores nulos en calificaciones:\n", calificaciones_df.isnull().sum())
print("Valores nulos en carrera:\n", carrera_df.isnull().sum())
print("Valores nulos en escuelas:\n", escuelas_df.isnull().sum())
print("Valores nulos en estudiantes:\n", estudiantes_df.isnull().sum())

# Verificar si hay duplicados
print("Duplicados en calificaciones:", calificaciones_df.duplicated().sum())
print("Duplicados en carrera:", carrera_df.duplicated().sum())
print("Duplicados en escuelas:", escuelas_df.duplicated().sum())
print("Duplicados en estudiantes:", estudiantes_df.duplicated().sum())

Valores nulos en calificaciones:
 periodo              0
clave_alumno         0
clave_materia        0
calificacion         0
tipo_calificacion    0
dtype: int64
Valores nulos en carrera:
 clave_carrera    0
nombre           0
corto            0
dtype: int64
Valores nulos en escuelas:
 clave      0
escuela    0
dtype: int64
Valores nulos en estudiantes:
 clave_alumno        0
clave_carrera       0
sexo                0
prepa               0
colonia           600
municipio         363
ciudad            287
ingreso             1
termino          5317
causa_salida        0
dtype: int64
Duplicados en calificaciones: 4
Duplicados en carrera: 0
Duplicados en escuelas: 0
Duplicados en estudiantes: 0


Limpieza de los datos

Eliminar duplicados:

In [148]:
# Eliminar duplicados
calificaciones_df = calificaciones_df.drop_duplicates()
carrera_df = carrera_df.drop_duplicates()
escuelas_df = escuelas_df.drop_duplicates()
estudiantes_df = estudiantes_df.drop_duplicates()

Manejo de valores nulos:

In [149]:
# Rellenar valores nulos con un valor específico (ejemplo: 0 o 'Desconocido')
calificaciones_df = calificaciones_df.fillna({'calificacion': 0, 'tipo_calificacion': 'Desconocido'})
carrera_df = carrera_df.fillna({'nombre': 'Desconocido', 'corto': 'Desconocido'})
escuelas_df = escuelas_df.fillna({'escuela': 'Desconocida'})

Reemplazar valores incorrectos o inconsistentes:

Si encuentras valores que no son correctos (por ejemplo, valores numéricos negativos donde no deberían ser), puedes reemplazarlos:

In [150]:
# Reemplazar valores negativos por 0 en calificación
calificaciones_df['calificacion'] = calificaciones_df['calificacion'].apply(lambda x: max(x, 0))


Unir los DataFrames en uno solo para hacer el análisis, usando merge():

In [151]:
# Verificar las columnas de los DataFrames
print(estudiantes_df.columns)
print(escuelas_df.columns)

# Unir estudiantes con carreras
estudiantes_calificaciones_escuelas = pd.merge(estudiantes_df, calificaciones_df, on='clave_alumno', how='left')
estudiantes_carrera = pd.merge(estudiantes_calificaciones_escuelas, carrera_df, on='clave_carrera', how='left')

# Unir con escuelas usando 'clave_carrera' de estudiantes y 'clave' de escuelas
estudiantes_completos = pd.merge(estudiantes_carrera, escuelas_df, left_on='prepa', right_on='clave', how='left')

estudiantes_completos = estudiantes_completos.rename(columns={
    'nombre': 'carrera_nombre',  # Renombrar columna 'nombre' de la tabla carrera
    'corto': 'carrera_nombre_corto',
    'clave': 'preparatoria_clave',
    
    'escuela': 'preparatoria_nombre'  # Renombrar columna 'escuela' de la tabla escuelas
})

# Verificar el resultado
display(estudiantes_completos)


Index(['clave_alumno', 'clave_carrera', 'sexo', 'prepa', 'colonia',
       'municipio', 'ciudad', 'ingreso', 'termino', 'causa_salida'],
      dtype='object')
Index(['clave', 'escuela'], dtype='object')


,clave_alumno,clave_carrera,sexo,prepa,colonia,municipio,ciudad,ingreso,termino,causa_salida,periodo,clave_materia,calificacion,tipo_calificacion,carrera_nombre,carrera_nombre_corto,preparatoria_clave,preparatoria_nombre
0,28,4,1,2,Los Angeles,25006.0,"CULIACAN, SIN.",2003.0,2061.0,5,2003.0,04tBt2,80.0,1.0,INGENIERIA ELECTRONICA,Electronica,NaN,NaN
1,28,4,1,2,Los Angeles,25006.0,"CULIACAN, SIN.",2003.0,2061.0,5,2003.0,04tBt1,88.0,1.0,INGENIERIA ELECTRONICA,Electronica,NaN,NaN
2,28,4,1,2,Los Angeles,25006.0,"CULIACAN, SIN.",2003.0,2061.0,5,2003.0,04tGt4,80.0,3.0,INGENIERIA ELECTRONICA,Electronica,NaN,NaN
3,28,4,1,2,Los Angeles,25006.0,"CULIACAN, SIN.",2003.0,2061.0,5,2003.0,04tGt5,88.0,1.0,INGENIERIA ELECTRONICA,Electronica,NaN,NaN
4,28,4,1,2,Los Angeles,25006.0,"CULIACAN, SIN.",2003.0,2061.0,5,2011.0,04tAt3,70.0,1.0,INGENIERIA ELECTRONICA,Electronica,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374857,11831,8,1,70,ADOLFO,25006.0,CULIACAN,2103.0,NaN,1,2103.0,TIF1019,77.0,1.0,INGENIERIA EN TECNOLOGIAS DE LA INFORMACION Y ...,TIC,70.0,CONALEP I (JUAN DE DIOS BATIZ- CULIACAN)
374858,11840,8,1,95,Las Mañanitas,25006.0,CULIACAN,2103.0,2103.0,4,2103.0,ACA0907,85.0,1.0,INGENIERIA EN TECNOLOGIAS DE LA INFORMACION Y ...,TIC,95.0,COBAC
374859,11840,8,1,95,Las Mañanitas,25006.0,CULIACAN,2103.0,2103.0,4,2103.0,ACC0906,83.0,1.0,INGENIERIA EN TECNOLOGIAS DE LA INFORMACION Y ...,TIC,95.0,COBAC
374860,11840,8,1,95,Las Mañanitas,25006.0,CULIACAN,2103.0,2103.0,4,2103.0,ACF0901,0.0,3.0,INGENIERIA EN TECNOLOGIAS DE LA INFORMACION Y ...,TIC,95.0,COBAC


Errores ortográficos en claves y datos categóricos:
- Normalizar cadenas:

In [152]:
estudiantes_df['ciudad'] = estudiantes_df['ciudad'].str.lower().str.strip()
estudiantes_df['colonia'] = estudiantes_df['colonia'].str.lower().str.strip()


Eliminar filas con valores nulos en columnas clave (calificación, materia, etc.)

In [153]:
# Eliminar filas con valores nulos en las columnas de calificaciones
estudiantes_completos_limpios = estudiantes_completos.dropna(subset=['periodo', 'clave_materia', 'calificacion', 'tipo_calificacion'])

# Guardar el DataFrame limpio
estudiantes_completos_limpios.to_csv('estudiantes_completos_limpios.csv', index=False)


Rellenar valores nulos con un valor predeterminado (por ejemplo, 0 para las calificaciones)

In [154]:
# Rellenar los valores nulos en las columnas clave con 0 o algún valor predeterminado
estudiantes_completos['calificacion'] = estudiantes_completos['calificacion'].fillna(0)
estudiantes_completos['periodo'] = estudiantes_completos['periodo'].fillna('Desconocido')  # O algún valor que se adapte
estudiantes_completos['clave_materia'] = estudiantes_completos['clave_materia'].fillna('Desconocido')
estudiantes_completos['tipo_calificacion'] = estudiantes_completos['tipo_calificacion'].fillna('Desconocido')

# Guardar el DataFrame limpio
estudiantes_completos.to_csv('estudiantes_completos_limpios.csv', index=False)


Solo verificar y visualizar las filas con valores nulos

In [155]:
# Ver las filas con valores nulos en las columnas clave
nulos = estudiantes_completos[estudiantes_completos.isnull().any(axis=1)]
print(nulos)

        clave_alumno  clave_carrera  sexo  prepa      colonia  municipio  \
0                 28              4     1      2  Los Angeles    25006.0   
1                 28              4     1      2  Los Angeles    25006.0   
2                 28              4     1      2  Los Angeles    25006.0   
3                 28              4     1      2  Los Angeles    25006.0   
4                 28              4     1      2  Los Angeles    25006.0   
...              ...            ...   ...    ...          ...        ...   
374853         11831              8     1     70       ADOLFO    25006.0   
374854         11831              8     1     70       ADOLFO    25006.0   
374855         11831              8     1     70       ADOLFO    25006.0   
374856         11831              8     1     70       ADOLFO    25006.0   
374857         11831              8     1     70       ADOLFO    25006.0   

                ciudad  ingreso  termino  causa_salida periodo clave_materia  \
0      

Verificación final y exportación del dataset

In [156]:
# Verificar si hay valores nulos en el DataFrame final
print(estudiantes_completos.isnull().sum())

# Guardar el DataFrame limpio
estudiantes_completos.to_csv('estudiantes_completos_limpios.csv', index=False)

clave_alumno                 0
clave_carrera                0
sexo                         0
prepa                        0
colonia                  14427
municipio                 8883
ciudad                    4754
ingreso                     52
termino                 133624
causa_salida                 0
periodo                      0
clave_materia                0
calificacion                 0
tipo_calificacion            0
carrera_nombre               0
carrera_nombre_corto         0
preparatoria_clave       66785
preparatoria_nombre      66785
dtype: int64


In [157]:
# Cargar el archivo CSV para visualizarlo
estudiantes_completos_guardados = pd.read_csv('estudiantes_completos_limpios.csv')

# Ver las primeras filas
display(estudiantes_completos_guardados)


/tmp/ipykernel_1961/3347554292.py:2: DtypeWarning: Columns (10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  estudiantes_completos_guardados = pd.read_csv('estudiantes_completos_limpios.csv')


,clave_alumno,clave_carrera,sexo,prepa,colonia,municipio,ciudad,ingreso,termino,causa_salida,periodo,clave_materia,calificacion,tipo_calificacion,carrera_nombre,carrera_nombre_corto,preparatoria_clave,preparatoria_nombre
0,28,4,1,2,Los Angeles,25006.0,"CULIACAN, SIN.",2003.0,2061.0,5,2003.0,04tBt2,80.0,1.0,INGENIERIA ELECTRONICA,Electronica,NaN,NaN
1,28,4,1,2,Los Angeles,25006.0,"CULIACAN, SIN.",2003.0,2061.0,5,2003.0,04tBt1,88.0,1.0,INGENIERIA ELECTRONICA,Electronica,NaN,NaN
2,28,4,1,2,Los Angeles,25006.0,"CULIACAN, SIN.",2003.0,2061.0,5,2003.0,04tGt4,80.0,3.0,INGENIERIA ELECTRONICA,Electronica,NaN,NaN
3,28,4,1,2,Los Angeles,25006.0,"CULIACAN, SIN.",2003.0,2061.0,5,2003.0,04tGt5,88.0,1.0,INGENIERIA ELECTRONICA,Electronica,NaN,NaN
4,28,4,1,2,Los Angeles,25006.0,"CULIACAN, SIN.",2003.0,2061.0,5,2011.0,04tAt3,70.0,1.0,INGENIERIA ELECTRONICA,Electronica,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374857,11831,8,1,70,ADOLFO,25006.0,CULIACAN,2103.0,NaN,1,2103.0,TIF1019,77.0,1.0,INGENIERIA EN TECNOLOGIAS DE LA INFORMACION Y ...,TIC,70.0,CONALEP I (JUAN DE DIOS BATIZ- CULIACAN)
374858,11840,8,1,95,Las Mañanitas,25006.0,CULIACAN,2103.0,2103.0,4,2103.0,ACA0907,85.0,1.0,INGENIERIA EN TECNOLOGIAS DE LA INFORMACION Y ...,TIC,95.0,COBAC
374859,11840,8,1,95,Las Mañanitas,25006.0,CULIACAN,2103.0,2103.0,4,2103.0,ACC0906,83.0,1.0,INGENIERIA EN TECNOLOGIAS DE LA INFORMACION Y ...,TIC,95.0,COBAC
374860,11840,8,1,95,Las Mañanitas,25006.0,CULIACAN,2103.0,2103.0,4,2103.0,ACF0901,0.0,3.0,INGENIERIA EN TECNOLOGIAS DE LA INFORMACION Y ...,TIC,95.0,COBAC


### Agrupar los datos para facilitar el analisis
se crean columnas de agregacion para analizar el desempeño global de cada alumno
facilitando la comparación entre estudiantes segun sus características

In [158]:
# Crear columnas de agregación
datos_agrupados = estudiantes_completos.groupby('clave_alumno').agg(
    materias_aprobadas=('calificacion', lambda x: (x >= 70).sum()),
    materias_reprobadas=('calificacion', lambda x: (x < 70).sum()),
    materias_totales=('calificacion', 'size'),
    promedio_calificaciones=('calificacion', 'mean'),
    desviacion_calificaciones=('calificacion', 'std'),
    duracion_estudios=('ingreso', lambda x: x.max() - x.min()),
    sexo=('sexo', 'first'),
    prepa=('preparatoria_clave', 'first'),
    prepa_nombre=('preparatoria_nombre', 'first'),
    carrera=('carrera_nombre_corto', 'first'),
    ciudad=('ciudad', 'first'),
    colonia=('colonia', 'first')
).reset_index()

# Revisar el resultado
display(datos_agrupados)


,clave_alumno,materias_aprobadas,materias_reprobadas,materias_totales,promedio_calificaciones,desviacion_calificaciones,duracion_estudios,sexo,prepa,prepa_nombre,carrera,ciudad,colonia
0,1,6,4,10,50.300000,43.815396,0.0,1,39.0,COBAES 39(PROFR. JOSE ROMERO ),Industrial,CULIACAN,Villa de Guadalupe
1,2,10,6,16,55.125000,45.070131,0.0,1,39.0,COBAES 39(PROFR. JOSE ROMERO ),Industrial,CULIACAN,Las Mañanitas
2,3,54,5,59,78.644068,25.825517,0.0,1,NaN,None,Industrial,CULIACAN,Morelos
3,4,48,16,64,58.765625,35.171571,0.0,2,36.0,COBAES 36(JOSE C.VALADEZ ROCH),Industrial,CUALIACAN,El Walamo
4,5,9,1,10,81.900000,29.990554,0.0,2,58.0,COBAES 58(ING.HERIBERTO VALDE),Industrial,CULIACAN,Las Granjas
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11931,11962,0,1,1,0.000000,NaN,0.0,1,22.0,COBAES 22(PROFR.MIGUEL CRISTO),Industrial,CULIACAN,4 de Marzo
11932,11963,0,1,1,0.000000,NaN,0.0,1,NaN,None,Mecatronica,None,None
11933,11964,0,1,1,0.000000,NaN,0.0,1,NaN,None,Sistemas,CULIACAN,Universidad 94
11934,11965,0,1,1,0.000000,NaN,0.0,1,NaN,None,Electrica,None,None


### Preprocesamiento para random forest
Crear variable categorica para campos no numericos que son necesarios en el analisis

In [159]:
# Codificación de variables categóricas
from sklearn.preprocessing import LabelEncoder

le_carrera = LabelEncoder()
le_ciudad = LabelEncoder()
le_colonia = LabelEncoder()

# Aplicamos la codificación
datos_agrupados['carrera_codificada'] = le_carrera.fit_transform(datos_agrupados['carrera'])
datos_agrupados['ciudad_codificada'] = le_ciudad.fit_transform(datos_agrupados['ciudad'])
datos_agrupados['colonia_codificada'] = le_colonia.fit_transform(datos_agrupados['colonia'])

display(datos_agrupados)

,clave_alumno,materias_aprobadas,materias_reprobadas,materias_totales,promedio_calificaciones,desviacion_calificaciones,duracion_estudios,sexo,prepa,prepa_nombre,carrera,ciudad,colonia,carrera_codificada,ciudad_codificada,colonia_codificada
0,1,6,4,10,50.300000,43.815396,0.0,1,39.0,COBAES 39(PROFR. JOSE ROMERO ),Industrial,CULIACAN,Villa de Guadalupe,3,137,784
1,2,10,6,16,55.125000,45.070131,0.0,1,39.0,COBAES 39(PROFR. JOSE ROMERO ),Industrial,CULIACAN,Las Mañanitas,3,137,510
2,3,54,5,59,78.644068,25.825517,0.0,1,NaN,None,Industrial,CULIACAN,Morelos,3,137,565
3,4,48,16,64,58.765625,35.171571,0.0,2,36.0,COBAES 36(JOSE C.VALADEZ ROCH),Industrial,CUALIACAN,El Walamo,3,125,267
4,5,9,1,10,81.900000,29.990554,0.0,2,58.0,COBAES 58(ING.HERIBERTO VALDE),Industrial,CULIACAN,Las Granjas,3,137,509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11931,11962,0,1,1,0.000000,NaN,0.0,1,22.0,COBAES 22(PROFR.MIGUEL CRISTO),Industrial,CULIACAN,4 de Marzo,3,137,17
11932,11963,0,1,1,0.000000,NaN,0.0,1,NaN,None,Mecatronica,None,None,6,545,870
11933,11964,0,1,1,0.000000,NaN,0.0,1,NaN,None,Sistemas,CULIACAN,Universidad 94,7,137,747
11934,11965,0,1,1,0.000000,NaN,0.0,1,NaN,None,Electrica,None,None,1,545,870


### Seleccion de caracteristicas y variable objetivo del random forest

In [160]:
# Selección de las características y el target
x = datos_agrupados[['ciudad_codificada', 'colonia_codificada', 
                           'sexo', 'carrera_codificada', 'prepa', ]]

# Variable objetivo: 'materias_aprobadas' (aprobado o reprobado)
y = datos_agrupados['materias_aprobadas']


### Entrenamiento del random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Crear el modelo de Random Forest
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Hacer predicciones
y_pred = rf_model.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Mostrar los resultados
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')
